In [1]:
from csoundengine import *

end of score.		   overall amps:      0.0
	   overall samples out of range:        0
0 errors in performance


In [2]:
s = Engine(latency=0.05).session()




--Csound version 6.19 (double samples) Apr 28 2024
[commit: 937aa495e250d758efaa28e9697a8b75237c80cd]
libsndfile-1.2.2
sr = 48000.0, kr = 750.000, ksmps = 64
0dBFS level = 1.0, A4 tuning = 440.0
audio buffered in 256 sample-frame blocks
reading 2048-byte blks of shorts from adc:Built-in Audio Pro: (RAW)
writing 512 sample blks of 64-bit floats to dac:Built-in Audio Pro:
SECTION 1:


In [3]:
tab = s.readSoundfile('../snd/piano-pitchtest-2.flac')
s.defInstr('test0', r'''
kfreq = p5
outch 1, oscili:a(0.1, lag(kfreq, 0.01))
''')
s.sched('test0', 0, 2, kfreq=440)
ev = s.playSample(tab)
ev

Button(description='Stop', style=ButtonStyle())

Output()

Synth(𝍪 .playSample=502.0002 start=23.204 dur=0.000 p4=301 isndtab=0 istart=0.02 ifadein=0.02 ifadeout=1 kchan=1 kspeed=1 kgain=0.5 kpan=0.02)

In [4]:
s.getInstr('.playSample')

Instr(.playSample, isndtab=0, istart=0, ifadein=0, ifadeout=0, kchan=1, speed(kspeed)=1, gain(kgain)=1, pan(kpan)=0.5, ixfade=-1)

In [5]:
s.defInstr('test1', r'''
|kfreq|
outch 1, oscili:a(0.05, lag(kfreq, 0.01))
''')


Instr(test1, kfreq=0)

In [6]:
name = 'test5'
s.defInstr(name, r'''
ibus = p5
kfreq = busin:k(ibus)
outch 1, oscili:a(0.05, lag(kfreq, 0.01))
''')


Instr(test5, ibus:5)

In [9]:
bus = s.assignBus(value=1000)
print(bus)
synth = s.sched(name, ibus=bus.token)
bus.automate((0, 800, 3, 500), overtake=False)
synth

Bus('control', token=1)


Button(description='Stop', style=ButtonStyle())

Output()

Synth(𝍪 test5=503.0003 start=43.859 dur=-1.000 p4=1)

In [10]:
synth.stop()

In [16]:
e = s.engine

buses = [e.assignBus(value=600) for _ in range(200)]
e.sync()
for bus in buses:
    synth = s.sched('test', dur=1, ibus=bus)
    e.writeBus(bus, 500, 0.001)
    

In [17]:
s.defInstr('test', r'''
  ibus = p5
  kfreq = busin:k(ibus)
  outch 1, oscili:a(0.05, lag(kfreq, 0.01))
''')

buses = [s.assignBus(value=2000) for _ in range(200)]
s.engine.sync()
for bus in buses:
    synth = s.sched('test', dur=1, ibus=bus.token)
    bus.set(500, 0)
    

In [27]:
e.writeBus(bus.token, 1500)

In [28]:
bus.set(1600)

In [18]:
e.unschedAll()

end of section 1	 sect peak amps:  0.87595  0.00000
SECTION 1:


In [19]:
s.defInstr('sender', r'''
ibus = p5
ifreqbus = p6
kfreq = busin:k(ifreqbus)
asig vco2 0.1, kfreq
busout(ibus, asig)
''')

s.defInstr('receiver', r'''
ibus  = p5
kgain = p6
asig = busin:a(ibus)
asig *= a(kgain)
outch 1, asig
''')


Instr(receiver, ibus:5, kgain:6)

In [20]:
bus = s.assignBus()
freqbus = s.assignBus(value=600)

chain = SynthGroup([s.sched('sender', ibus=bus, ifreqbus=freqbus),
                    s.sched('receiver', ibus=bus, kgain=0.5)])
e.writeBus(freqbus.token, 400, delay=1)
chain

Button(description='Stop', style=ButtonStyle())

Output()

p1,start,dur,p4,ibus,ifreqbus
505.0002 ▶,-0.125,-1.000,0,1202,1203
p1,start,dur,p4,ibus,kgain
506.0002 ▶,-0.061,-1.000,0,1202,0.5
